In [1]:
import requests 

In [2]:
import pandas as pd

In [3]:
from bs4 import BeautifulSoup

In [4]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [22]:
years = list(range(2024,2020,-1))

In [23]:
years

[2024, 2023, 2022, 2021]

In [24]:
from io import StringIO

In [25]:
all_matches = []

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    time.sleep(12)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        time.sleep(12)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try: 
          team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name 
        all_matches.append(team_data)
        time.sleep(12)

C:\Users\ShriyaAsija\AppData\Local\Temp\ipykernel_16916\2191145324.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\ShriyaAsija\AppData\Local\Temp\ipykernel_16916\2191145324.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\ShriyaAsija\AppData\Local\Temp\ipykernel_16916\2191145324.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\ShriyaAsija\AppData\Local\Temp\ipykernel_16916\2191145324.py:26: FutureWarning: Passing 

In [ ]:
match_df = pd.concat(all_matches)

In [29]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2020-09-21,20:15,Premier League,Matchweek 2,Mon,Away,W,3,1,Wolves,...,Match Report,NaN,13.0,8.0,21.1,2.0,1,1,2024,Manchester City
2,2020-09-27,16:30,Premier League,Matchweek 3,Sun,Home,L,2,5,Leicester City,...,Match Report,NaN,16.0,5.0,19.8,1.0,0,0,2024,Manchester City
4,2020-10-03,17:30,Premier League,Matchweek 4,Sat,Away,D,1,1,Leeds United,...,Match Report,NaN,23.0,1.0,18.2,1.0,0,0,2024,Manchester City
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1,0,Arsenal,...,Match Report,NaN,13.0,5.0,17.7,0.0,0,0,2024,Manchester City
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1,1,West Ham,...,Match Report,NaN,14.0,7.0,20.9,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2018-04-15,16:00,Premier League,Matchweek 34,Sun,Away,W,1,0,Manchester Utd,...,Match Report,NaN,10.0,4.0,18.1,0.0,0,0,2021,West Bromwich Albion
39,2018-04-21,12:30,Premier League,Matchweek 35,Sat,Home,D,2,2,Liverpool,...,Match Report,NaN,13.0,6.0,17.7,0.0,0,0,2021,West Bromwich Albion
40,2018-04-28,15:00,Premier League,Matchweek 36,Sat,Away,W,1,0,Newcastle Utd,...,Match Report,NaN,9.0,2.0,20.1,0.0,0,0,2021,West Bromwich Albion
41,2018-05-05,15:00,Premier League,Matchweek 37,Sat,Home,W,1,0,Tottenham,...,Match Report,NaN,9.0,1.0,10.2,0.0,0,0,2021,West Bromwich Albion


In [30]:
match_df.columns = [c.lower() for c in match_df.columns]

In [31]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2020-09-21,20:15,Premier League,Matchweek 2,Mon,Away,W,3,1,Wolves,...,Match Report,NaN,13.0,8.0,21.1,2.0,1,1,2024,Manchester City
2,2020-09-27,16:30,Premier League,Matchweek 3,Sun,Home,L,2,5,Leicester City,...,Match Report,NaN,16.0,5.0,19.8,1.0,0,0,2024,Manchester City
4,2020-10-03,17:30,Premier League,Matchweek 4,Sat,Away,D,1,1,Leeds United,...,Match Report,NaN,23.0,1.0,18.2,1.0,0,0,2024,Manchester City
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1,0,Arsenal,...,Match Report,NaN,13.0,5.0,17.7,0.0,0,0,2024,Manchester City
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1,1,West Ham,...,Match Report,NaN,14.0,7.0,20.9,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2018-04-15,16:00,Premier League,Matchweek 34,Sun,Away,W,1,0,Manchester Utd,...,Match Report,NaN,10.0,4.0,18.1,0.0,0,0,2021,West Bromwich Albion
39,2018-04-21,12:30,Premier League,Matchweek 35,Sat,Home,D,2,2,Liverpool,...,Match Report,NaN,13.0,6.0,17.7,0.0,0,0,2021,West Bromwich Albion
40,2018-04-28,15:00,Premier League,Matchweek 36,Sat,Away,W,1,0,Newcastle Utd,...,Match Report,NaN,9.0,2.0,20.1,0.0,0,0,2021,West Bromwich Albion
41,2018-05-05,15:00,Premier League,Matchweek 37,Sat,Home,W,1,0,Tottenham,...,Match Report,NaN,9.0,1.0,10.2,0.0,0,0,2021,West Bromwich Albion


In [33]:
match_df.to_csv("matches.csv")